In [ ]:
%%bash

sudo apt install build-essential -y
conda install -c nvidia cuda
conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch
pip install ninja clean-fid wandb lpips dominate Pillow
pip install -r requirements.txt

pip install gpp # Don't need this one, gpp is not found in pip.

In [ ]:
!sudo ln -s /home/lingz/anaconda3/lib/libcudart.so /usr/lib/libcudart.so

In [ ]:
conda install -c conda-forge pytorch nvidia

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3, 1), 'GB')


In [5]:
!python generate.py --samples 5

End / Spend: 3.378163 sec


In [ ]:
!python train.py \
--name animate --size 256 --batch 1 --max_iter 10 --l_image 0.7 \
--dataroot_sketch "./input/sketch" --d_pretrained "./pretrained/d_net.pth" \
--dataroot_image "./input/image" --g_pretrained "./pretrained/g_net.pth" \
--disable_eval --no_wandb --diffaug_policy translation

In [ ]:
!python train.py \
--name animate --batch 1 --max_iter 10 --disable_eval --no_wandb \
--dataroot_sketch "./input/sketch" --dataroot_image "./input/image" \

In [5]:
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from pathlib import Path
import smtplib

content = MIMEMultipart()
content["subject"] = "Easy Paint Anime!"
content["from"] = "Easy Paint Anime! Work Team"
content["to"] = "lingziha@gmail.com"
content.attach(MIMEImage(Path("output/000001.png").read_bytes()))


with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:
    try:
        smtp.ehlo()
        smtp.starttls()
        smtp.login("08360716@me.mcu.edu.tw", "oxqnefmwnrzaajwb")
        smtp.send_message(content)
        print("Complete!")
    except Exception as e:
        print("Error message: ", e)

Complete!


In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP

# 要加密的資料（必須為 bytes）
data = b'My secret data.'

# 讀取 RSA 公鑰
publicKey = RSA.import_key(open("public.pem").read())

# 建立隨機的 AES Session 金鑰
sessionKey = get_random_bytes(16)

# 以 RSA 金鑰加密 Session 金鑰
cipherRSA = PKCS1_OAEP.new(publicKey)
encSessionKey = cipherRSA.encrypt(sessionKey)

# 以 AES Session 金鑰加密資料
cipherAES = AES.new(sessionKey, AES.MODE_EAX)
ciphertext, tag = cipherAES.encrypt_and_digest(data)

# 將加密結果寫入檔案
with open("encrypted_data.bin", "wb") as f:
    f.write(encSessionKey)
    f.write(cipherAES.nonce)
    f.write(tag)
    f.write(ciphertext)

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP

# 讀取 RSA 私鑰
privateKey = RSA.import_key(open("private.pem").read())

# 從檔案讀取加密資料
with open("encrypted_data.bin", "rb") as f:
    encSessionKey = f.read(privateKey.size_in_bytes())
    nonce = f.read(16)
    tag = f.read(16)
    ciphertext = f.read(-1)

# 以 RSA 金鑰解密 Session 金鑰
cipherRSA = PKCS1_OAEP.new(privateKey)
sessionKey = cipherRSA.decrypt(encSessionKey)

# 以 AES Session 金鑰解密資料
cipherAES = AES.new(sessionKey, AES.MODE_EAX, nonce)
data = cipherAES.decrypt_and_verify(ciphertext, tag)

# 輸出解密後的資料
print(data.decode("utf-8"))